<h1>Cyberbullying detection</h1>
<hr>
<h3>Preprocessing</h3>
<hr>

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import re


<hr>
read the dataset
<hr>

In [54]:
df_cbd = pd.read_csv('../datasets/twitter_parsed_dataset_post_EDA.csv')

In [55]:
df_cbd.shape

(16851, 5)

Drop those 3 rows with null values detected on EDA stage.

In [56]:
df_cbd[df_cbd['id'].isnull() | df_cbd['text'].isnull() | df_cbd['annotation'].isnull()] 

,index,id,text,annotation,oh_label
2992,5.67909969783169E+017,5.67909969783169E+017,RT @Lizzethismetal: Probably gonna piss off so...,NaN,NaN
2993,#yesallmen,NaN,NaN,NaN,NaN
2994,@MT8_9 http://t.co/QHo6dhumjc,sexism,1,NaN,NaN


In [57]:
df_cbd.drop(df_cbd[df_cbd['id'].isnull() | df_cbd['text'].isnull() | df_cbd['annotation'].isnull()].index,axis=0,inplace=True)
df_cbd.shape

(16848, 5)

Start to remove some text like: http tag, punctuations, special characters

In [58]:
#remove url's
df_cbd['text'] = df_cbd['text'].map(lambda x : re.sub(r"\S*https?:\S*", '', x))

In [59]:
#remove RT mark
df_cbd['text'] =df_cbd['text'].map(lambda x : re.sub(r"^RT @*", '', x))

In [60]:
#remove punctuaction marks and special characters
df_cbd['text']=df_cbd['text'].map(lambda x : re.sub(r"[!\"#$%&\'()*+,-./:;=>?@[\]^_`{|}~]*", '', x))

In [61]:
#let's check if I have success 😂
df_cbd.head(30)

,index,id,text,annotation,oh_label
0,5.74948705591165E+017,5.74948705591165E+017,halalflaws biebervalue greenlinerzjm I read th...,none,0.0
1,5.71917888690393E+017,5.71917888690393E+017,ShreyaBafna3 Now you idiots claim that people ...,none,0.0
2,3.90255841338601E+017,3.90255841338601E+017,Mooseoftorment Call me sexist but when I go to...,sexism,1.0
3,5.68208850655916E+017,5.68208850655916E+017,g0ssipsquirrelx Wrong ISIS follows the example...,racism,1.0
4,5.75596338802373E+017,5.75596338802373E+017,mkr No No No No No No,none,0.0
5,5.61184804004041E+017,5.61184804004041E+017,TRobinsonNewEra Saudi preacher who raped and ...,none,0.0
6,4.36563082093949E+017,4.36563082093949E+017,Millhouse66 MaureenJS nooo not sexist but most...,sexism,1.0
7,5.75962326454641E+017,5.75962326454641E+017,Going to make some pancakesDont hve any strawb...,none,0.0
8,5.72581510248706E+017,5.72581510248706E+017,ahtweet freebsdgirl How dare you have feelings...,none,0.0
9,5.00456367719989E+017,5.00456367719989E+017,Newmanzaa Theres something wrong when a girl w...,sexism,1.0


In [ ]:
#let's remove the stopwords 